# Classification model for Data Intensive course

1. **Descrizione del problema e analisi del dataset**: descrivere contesto e obiettivi del modello di classificazione, descrivere il dataset e le variabili contenute..

2. **Caricamento e preparazione dei dati**: descrivere come i dati sono stati caricati e preparati per il modello, inclusi eventuali passaggi di pulizia o trasformazione dei dati. Descrivere le features.

3. **Analisi esplorativa delle features**: descrivere le principali caratteristiche del dataset, inclusi statistiche descrittive, distribuzioni e correlazioni tra le variabili. Commentare i risultati ottenuti.

4. **Preparare i dati**: prepariamo i dati per l'addestramento del modello e facciamo preprocessing. Questo include la normalizzazione delle feature, la gestione dei valori mancanti e la suddivisione del dataset in set di addestramento e di test, eseguire eventuali operazioni di
preprocessing come ad es. one-hot encoding di variabili categoriche e oversampling o undersampling in
caso di classi sbilanciate.

5. **Addestramento dei modelli**: addestriamo diversi modelli di classificazione sui dati preparati e valutiamo le loro prestazioni utilizzando metriche appropriate. (Testare due famiglie diverse di algoritmi)

6. **Analisi dei risultati**: analizziamo i risultati ottenuti dai modelli addestrati, confrontando le loro prestazioni e identificando il modello migliore per il nostro problema di classificazione.

7. **Ricerca degli iperparametri**: ottimizziamo gli iperparametri dei modelli utilizzando tecniche come la randomized search o la grid search per migliorare le loro prestazioni.

8. **Conclusioni**: riassumiamo i risultati ottenuti e discutiamo le implicazioni del nostro lavoro, evidenziando i punti di forza e le limitazioni del modello di classificazione sviluppato.